# How to Capture New Knowledge for Your Bug Tracker

Congratulations!
The Bug Tracker you designed for your company is a hit!
Now, how do we take the data we have and turn it into some meaningful metrics.

### Retreiving the Cloud SQL MySQL Database Host

From the [GCP Console](https://console.cloud.google.com/) search bar at the top of the page, type in SQL. Click the SQL result to open the Cloud SQL console view.

![](assets/20250423102654.png)

Observe the bugtracker-db Cloud SQL MySQL instance that has been created for you by the Cloud Academy lab environment.

Continue with the lab once the instance is ready. It can take 20 minutes from when you started the lab for the instance to reach the ready status:

![](assets/20250423103415.png)

Copy the Public IP address and use it to overwrite the `REPLACE_ME` value in the code block in the following section.

### Populating the Database

Before diving in, we'll need to populate the database with some mock data.
Use the `data.sql` file along with the python code below to create and populate the database.
We'll be creating 20 users and 1000 tickets.
That should be enough data to work with.

In [ ]:
# create database and table schema
! pip install mysql-connector-python

import mysql.connector

# make sure the host matches your GCP instance along with the password created earlier.
host = "REPLACE_ME"
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123"
)

cursor = db.cursor()

# drop/create/insert database
with open('./data_files/lab9/data.sql') as f:
    commands = f.read().split(';')
    f.close()
    print('populating database...')
    for command in commands:
        try:
            if command.strip() != '':
                cursor.execute(command)
        except:
            print('[ERROR]:', command)
    db.commit()
    print('data insert complete')

cursor.close()
db.close()

Let's run a small test query to make sure the data was populated correctly.
The query below will grab a ticket id, ticket title, and the users email address that is assigned to the ticket.
The `title` might not make any sense, but thats what test data is for!

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor()
# cursor.execute('SELECT t.id, t.title, u.email FROM tickets t INNER JOIN users u ON t.assignee=u.id ORDER BY RAND() LIMIT 5')
cursor.execute('SELECT t.id, t.title, u.email FROM tickets t INNER JOIN users u ON t.assignee=u.id ORDER BY RAND() LIMIT 5')
print(*cursor.fetchall(), sep='\n')

cursor.close()
db.close()

Here's the CREATE query for the `users` and `tickets` tables in our database.

```sql
CREATE TABLE IF NOT EXISTS users (
  id INT AUTO_INCREMENT PRIMARY KEY,
  first_name VARCHAR(30),
  last_name VARCHAR(30),
  email VARCHAR(80) UNIQUE
) ENGINE=InnoDB;

CREATE TABLE IF NOT EXISTS tickets (
  id INT AUTO_INCREMENT PRIMARY KEY,
  title VARCHAR(128),
  project VARCHAR(64) NOT NULL,
  assignee INT,
  description TEXT,
  date_created DATETIME DEFAULT CURRENT_TIMESTAMP,
  date_updated DATETIME DEFAULT CURRENT_TIMESTAMP,
  date_completed DATETIME NULL DEFAULT NULL,
  FOREIGN KEY fk_assignee_id(assignee) REFERENCES users(id)
) ENGINE=InnoDB;
```

We have all this data, and we need some meaningful metrics.
We need to ask ourselves: _what question are we trying to answer with this data?_
Throughout this lab, we'll learn how to answer the following questions:

- How many tickets are open/closed?
- Who has the most tickets assigned?
- What is the longest time taken to complete a task?
- What is the average time to close a ticket?
- How many tickets we're completed in less than 10 days?

There are a few ways to answer these questions.
One could query the database for individual data, and then compare and/or manipulate it using python.
Thoughout this lab we'll be doing all of the computations within MySQL, and python will only be our conduit to execute our queries.
To do this, we'll be using MySQL's [aggregate functions](https://en.wikipedia.org/wiki/Aggregate_function).
These are common in every RDBMS, such as: `SUM()`, `COUNT()`, and `AVG()`, to name a few.
We'll be using these to answer the questions asked earlier.

## Question 1: _How many tickets are open/closed?_

Looking at the `tickets` table, the `date_created` column does not have a default value.
This is used to determine if a ticket is closed or not.
If the colum is `NULL` then the ticket is considered open and considered closed when any date value is populated.
Since we only care if the column is populated, not the data inself, we can use the `COUNT` aggregate function.
The `COUNT` function counts the number of rows that it matches.

Let's craft a SQL query to count the number of open tickets.
To begin we'll count all tickets using `COUNT(*)`.
```sql
SELECT COUNT(*) AS counted_tickets
FROM tickets t
```
To narrow this down we'll use the `WHERE` clause to only find tickets with `date_completed` being populated, giving us the number of closed tickets.
```sql
SELECT COUNT(*) AS closed_tickets
FROM tickets t
WHERE t.date_completed IS NOT NULL
```
We'll do the opposite for open tickets using `WHERE t.date_completed IS NULL`.
Refer to the python code below for a full example.

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# get number of tickets open
cursor.execute('SELECT COUNT(*) AS tickets_open FROM tickets t WHERE t.date_completed IS NULL')
tickets_open = cursor.fetchone()['tickets_open']

# get number of tickets closed
cursor.execute('SELECT COUNT(*) AS tickets_closed FROM tickets t WHERE t.date_completed IS NOT NULL')
tickets_closed = cursor.fetchone()['tickets_closed']

print('Tickets Open: {}\nTickets Closed: {}'.format(tickets_open, tickets_closed))

cursor.close()
db.close()

### Using a Single Query
What if we wanted to combine the queries above into a single query to count the tickets in the database.
There's nothing wrong with the code above, this is just another example of how to solve the same problem.
The is not as straight forward as you might expect.
One cannot simply combine the `SELECT` statements.

For this to work correctly, we'll need to use some trickery using the `SUM` aggregate function, along with a `CASE` statement.
A `CASE` statement is is the same as [switch statement](https://en.wikipedia.org/wiki/Switch_statement) if your familar with other programming languages.
The trick is to return either a 1 or a 0 when `date_completed` is `NULL`.
This can be added into the `SELECT` statement as such:
```sql
SUM(CASE WHEN t.date_completed IS NULL THEN 1 ELSE 0 END) AS 'tickets_open',
```

To calcuate tickets_closed, swap the 1 and 0 for the same logic:
```sql
SELECT
  SUM(CASE WHEN t.date_completed IS NULL THEN 1 ELSE 0 END) AS 'tickets_open',
  SUM(CASE WHEN t.date_completed IS NULL THEN 0 ELSE 1 END) AS 'tickets_closed'
FROM tickets t;
```

#### Exercise
Using the SQL above, edit the python code to use the single query and print the results to the console. (The solutions to exercises are included at the end of this notebook if you need to consult them)

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# Enter the single line query SQL below using cursor.execute()

result = cursor.fetchone()

# get number of tickets open
tickets_open = result['tickets_open']

# get number of tickets closed
tickets_closed = result['tickets_closed']

print('Tickets Open: {}\nTickets Closed: {}'.format(tickets_open, tickets_closed))

cursor.close()
db.close()

## Question 2: _Who has the most tickets assigned?_

Explain `SUM()`, joining, `GROUP BY`, `ORDER BY`, and `LIMIT`
To go about answering this question, we'll again turn to the `COUNT` function.
As we need to count the number of tickets for each `assignee`, we'll also want to view the assignee in question.
Let's start by querying for the `assignee` and counting the `tickets.id` column.
To make this work, we need to [`GROUP BY`](https://dev.mysql.com/doc/refman/8.0/en/group-by-modifiers.html) the `assignee`.
```sql
SELECT t.assignee, COUNT(t.id) AS ticket_count FROM tickets t GROUP BY t.assignee;
```
This results in displaying the `assignee` along with the number of tickets they are assigned to.
The result set looks similar to the following.

![ticket count results](./assets/lab9/ticket_count_1.png)

There are a couple of things wrong here:

1. The ticket count is out of order, and its hard to determine who has the most.
2. We get _all_ the users and their tickets, but we only want the user with the most tickets.
3. Showing the user.id doesn't give us much information about ***who*** has the most tickets.

Let's take care of each issue, one at a time.

First, to order the results, we'll use the `ORDER BY` statement.
This requires one or more columns to determine the order.
By default, it will order the results in ascending order, we can change that by added `DESC` for descending after the column name.
Let's update the SQL query:

```sql
SELECT t.assignee, COUNT(t.id) AS ticket_count FROM tickets t GROUP BY t.assignee ORDER BY ticket_count DESC;
```

Second, we only want the top result, we can tell MySQL to only return the first result by using the `LIMIT` statement.
This returns `n` number of rows as a result.
Since we're in descending order, let's limit this to the first result.

```sql
SELECT t.assignee, COUNT(t.id) AS ticket_count
FROM tickets t
GROUP BY t.assignee
ORDER BY ticket_count DESC
LIMIT 1;
```
Finally, let's add a little more information about the user.
We'll do a `JOIN` on the user table and include the user's name along with their email address.

```sql
SELECT
    CONCAT(u.first_name, ' ', u.last_name) AS name,
    u.email, 
    COUNT(t.id) AS ticket_count
FROM tickets t
INNER JOIN users u ON u.id=t.assignee 
GROUP BY t.assignee 
ORDER BY ticket_count DESC
LIMIT 1;
```
The result will look similar to the following.

![ticket count results](./assets/lab9/ticket_count_2.png)

Below is a python script to execute the query and print the results.

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

query = '''
SELECT
    CONCAT(u.first_name, ' ', u.last_name) AS name,
    u.email, 
    COUNT(t.id) AS ticket_count
FROM tickets t
INNER JOIN users u ON u.id=t.assignee 
GROUP BY t.assignee 
ORDER BY ticket_count DESC
LIMIT 1;
'''

# execute the query
cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

### Exercise

Update the SQL query to determine who has the _most closed tickets_ and who has the _most open tickets_.

## Question 3: _What is the longest time taken to complete a task?_

Now we're going to start using data that we don't directly have in our database.
We don't have a column called `days_to_complete` that we can query, but we can derive that using columns within our table.
To determine how many days it takes to complete a ticket, we can use the [`DATEDIFF` (MySQL manual)](https://dev.mysql.com/doc/refman/8.0/en/date-and-time-functions.html#function_datediff) function on `tickets.date_created` and `tickets.date_completed`.
While these columns are `TIMESTAMPS` we're only interested in how many _days_ it takes, and `DATEDIFF` returns the difference in days.

This will use a lot of the techniques we've covered already.
We'll need to make sure `date_completed` is `NOT NULL` as we're only focused on completed tickets; we'll need to order results in descending order; and limit the results to the first row.

Before viewing the SQL code below, try to type out the SQL command yourself.

```sql
SELECT
    DATEDIFF(t.date_completed, t.date_created) AS completed
FROM tickets t
WHERE t.date_completed IS NOT NULL
ORDER BY completed DESC
LIMIT 1;
```

You should get a result similar to the following:

![logest task](./assets/lab9/long_task.png)

The results can be acheived using the python code below.

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT
    DATEDIFF(t.date_completed, t.date_created) AS completed
FROM tickets t
WHERE t.date_completed IS NOT NULL
ORDER BY completed DESC
LIMIT 1
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

## Question 4: _- What is the average time to close a ticket?_

Answering this question builds of the previous question.
However, this time, we'll use the `AVG` aggregate function.
This takes the average of all the data of a specific column.
Be sure to remove `LIMIT 1` from the SQL query!

```sql
SELECT
    AVG( DATEDIFF(t.date_completed, t.date_created) ) AS avg_completion_time
FROM tickets t 
WHERE t.date_completed IS NOT NULL;
```

The python code to run and display the query.

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT
    AVG( DATEDIFF(t.date_completed, t.date_created) ) AS avg_completion_time
FROM tickets t 
WHERE t.date_completed IS NOT NULL
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

### Exercise

Notice how the result has four decimal places?
Use python to format the result to two decimal places.
How could you achieve the same result using MySQL?

## Question 5: _How many tickets were completed in less than 10 days?_

First we need to create a query that displays the `ticket.id` and time it took to complete.
We'll build off the previous questions to construct this.

```sql
SELECT t.id, DATEDIFF(t.date_completed, t.date_created) AS completed FROM tickets t WHERE t.date_completed IS NOT NULL
```

![id and days completed](./assets/lab9/ten_days_1.png)

Next, we'll need to limit the results to only `tickets.id` that took less than 10 days to complete.
This is different than the `LIMIT` clause used earlier.
For this we'll be using the [`HAVING` statement](https://en.wikipedia.org/wiki/Having_(SQL)).
This is similar to the `WHERE` clause, but the key difference is that it relates to the `SELECT` results, instead of the data within the table.

```sql
SELECT t.id, DATEDIFF(t.date_completed, t.date_created) AS completed FROM tickets t WHERE t.date_completed IS NOT NULL
HAVING completed <= 10
```

![id and days completed](./assets/lab9/ten_days_2.png)

This is looking much better.
We have the data we want, but we need a way to count it.
Unfortunately, we cannot just wrap the whole `SELECT` statement in a `COUNT` function.
Although, we can get pretty close to that.

We'll be using a [subquery](https://en.wikipedia.org/wiki/SQL_syntax#Subqueries) to build an anonymous _table_ of the data we want and then use that data to query from.
This is done by wrapping the query above in parentheses in the `FROM` clause:

```sql
SELECT * FROM (
    SELECT t.id, DATEDIFF(t.date_completed, t.date_created) AS completed
    FROM tickets t
    WHERE t.date_completed IS NOT NULL
    HAVING completed <= 10
) AS quickly_completed_tickets;
```

![id and days completed in subquery](./assets/lab9/ten_days_3.png)

Notice how the data returned is _exactly the same_ as the previous data set.
Now we can use the `COUNT` function to get our final result, the number of tickets that were completed within 10 days.

```sql
SELECT COUNT(*) ticket_count 
FROM (
    SELECT
        t.id,
        DATEDIFF(t.date_completed, t.date_created) AS completed
    FROM tickets t 
    WHERE t.date_completed IS NOT NULL 
    HAVING completed <= 10
) AS quickly_completed_tickets
```

![the final result](./assets/lab9/ten_days_4.png)

And finally, the python code to run the query!

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT COUNT(*) ticket_count 
FROM (
    SELECT
        t.id,
        DATEDIFF(t.date_completed, t.date_created) AS completed
    FROM tickets t 
    WHERE t.date_completed IS NOT NULL 
    HAVING completed <= 10
) AS quickly_completed_tickets
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close() 

### Exercise

Find tickets that took _more than 80 days_ to complete.
How can you find tickets that were updated before July 10th, 2020 but not more than one month before?

---

**BONUS**: Ask a question about the data, and then answer it using Python and SQL.

## Solutions to exercises

### How many tickets are open/closed: Using a Single Query

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

cursor.execute("SELECT SUM(CASE WHEN t.date_completed IS NULL THEN 1 ELSE 0 END) AS 'tickets_open', SUM(CASE WHEN t.date_completed IS NULL THEN 0 ELSE 1 END) AS 'tickets_closed' FROM tickets t")

result = cursor.fetchone()

# get number of tickets open
tickets_open = result['tickets_open']

# get number of tickets closed
tickets_closed = result['tickets_closed']

print('Tickets Open: {}\nTickets Closed: {}'.format(tickets_open, tickets_closed))

cursor.close()
db.close()


### Update the SQL query to determine who has the most closed tickets and who has the most open tickets

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

query = '''
SELECT
    CONCAT(u.first_name, ' ', u.last_name) AS name,
    u.email, 
    SUM(CASE WHEN t.date_completed IS NOT NULL THEN 1 ELSE 0 END) AS closed_tickets
FROM tickets t
INNER JOIN users u ON u.id=t.assignee 
GROUP BY t.assignee 
ORDER BY closed_tickets DESC
LIMIT 1;
'''

# execute the query
cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

### How could you format the result to two decimal places using MySQL?

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT
    TRUNCATE( AVG( DATEDIFF(t.date_completed, t.date_created) ), 2) AS avg_completion_time
FROM tickets t 
WHERE t.date_completed IS NOT NULL
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()

### Find tickets that took more than 80 days to complete

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
SELECT COUNT(*) ticket_count 
FROM (
    SELECT
        t.id,
        DATEDIFF(t.date_completed, t.date_created) AS completed
    FROM tickets t 
    WHERE t.date_completed IS NOT NULL 
    HAVING completed > 80
) AS quickly_completed_tickets
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()


### How to find tickets that were updated before July 10th, 2020 but not more than one month before

In [ ]:
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker_aggregate"
)

cursor = db.cursor(dictionary=True)

# query to answer the question
query = '''
    SELECT *
    FROM tickets
    WHERE date_updated > '2020-06-10' AND date_updated < '2020-07-10'
'''

cursor.execute(query)

print(*cursor.fetchall(), sep='\n')
cursor.close()
db.close()
